### imports

In [1]:
import pandas as pd
import numpy as np

# Load Data

In [2]:
weatherData_raw = pd.read_sql_table('WeatherData', 'sqlite:///data.sqlite')
crashData2017_raw = pd.read_sql_table('CrashData2017', 'sqlite:///data.sqlite')
crashData2018_raw = pd.read_sql_table('CrashData2018', 'sqlite:///data.sqlite')
crashData2019_raw = pd.read_sql_table('CrashData2019', 'sqlite:///data.sqlite')
weatherData = weatherData_raw.copy()

# Filter CrashData to match time frame of WeatherData

In [3]:
crashData2017 = crashData2017_raw[crashData2017_raw['UMONAT'] == 12]
crashData2019 = crashData2019_raw[crashData2019_raw['UMONAT'] != 12]
crashData2018 = crashData2018_raw.copy()

#### Remove bycicle and pedestrian accidents as they are not relevant for the analysis, only highway accidents are relevant

In [4]:
crashData2017 = crashData2017[crashData2017['IstRad'] != 1]
crashData2018 = crashData2018[crashData2018['IstRad'] != 1]
crashData2019 = crashData2019[crashData2019['IstRad'] != 1]
crashData2017 = crashData2017[crashData2017['IstFuss'] != 1]
crashData2018 = crashData2018[crashData2018['IstFuss'] != 1]
crashData2019 = crashData2019[crashData2019['IstFuss'] != 1]

#### Remove accidents that fall in the categorie "Einbiegen / Kreuzen-Unfall" and "Zusammenstoß zwischen Fahrzeug und Fußgänger" as they are not relevant for the analysis, only accidents that happen on the highway are relevant

In [5]:
crashData2017 = crashData2017[crashData2017['UTYP1'] != 3]
crashData2018 = crashData2018[crashData2018['UTYP1'] != 3]
crashData2019 = crashData2019[crashData2019['UTYP1'] != 3]
crashData2017 = crashData2017[crashData2017['UART'] != 6]
crashData2018 = crashData2018[crashData2018['UART'] != 6]
crashData2019 = crashData2019[crashData2019['UART'] != 6]

#### Merge column IstSonstig and IstGkfz into IstSonstig as in 2017 there was no column IstGkfz

In [6]:
crashData2018['IstSonstig'] = crashData2018['IstSonstig'] | crashData2018['IstGkfz']
crashData2019['IstSonstige'] = crashData2019['IstSonstige'] | crashData2019['IstGkfz']
crashData2019 = crashData2019.rename(columns={"IstSonstige": "IstSonstig"})

#### Remove columns that are not relevant for the analysis

In [7]:
crashData2017 = crashData2017.drop(['LINREFX', 'LINREFY', 'IstRad', 'IstFuss', 'OBJECTID', 'UIDENTSTLA'], axis=1)
crashData2018 = crashData2018.drop(['LINREFX', 'LINREFY', 'IstRad', 'IstFuss', 'OBJECTID_1'], axis=1)
crashData2019 = crashData2019.drop(['LINREFX', 'LINREFY', 'IstRad', 'IstFuss', 'OBJECTID'], axis=1)

#### Reorder columns

In [8]:
# Get the column order from df1
column_order = crashData2017.columns

# Reorder the columns in df2 based on the column order from df1
crashData2018 = crashData2018.reindex(columns=column_order)
crashData2019 = crashData2019.reindex(columns=column_order)

# Match Coordinates of CrashData to WeatherData to select only relevant Crashdata

In [9]:
def calculate_distance(coords1, coords2):
    # Radius of the Earth in meters
    radius = 6371000

    # Convert latitudes and longitudes to radians
    lat1_rad = np.radians(coords1[:, 0])
    lon1_rad = np.radians(coords1[:, 1])
    lat2_rad = np.radians(coords2[:, 0])
    lon2_rad = np.radians(coords2[:, 1])

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = radius * c
    return distance

# Set a threshold distance for considering two locations as a match
threshold_distance = 600  # 600 meters as the weather points are 1km apart and we want to match the closest one

In [10]:
# Calculate distances using a loop
distances = []
for i, row in crashData2017.iterrows():
    lat1, lon1 = row['Latitude'], row['Longitude']
    dist = calculate_distance(np.array([(lat1, lon1)]), weatherData[['Latitude', 'Longitude']].to_numpy())
    distances.append(dist)

# Find matching rows
distances = np.concatenate(distances, axis=0)
distances = distances.reshape(len(crashData2017), len(weatherData))
crashData2017_nearby = crashData2017[np.min(distances, axis=1) <= threshold_distance]

crashData2017_nearby.head()

,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,UTYP1,IstPKW,IstKrad,IstSonstig,LICHT,STRZUSTAND,Longitude,Latitude
8294,2,1,15,134,2017,12,17,4,3,2,6,1,0,0,2,1,10.023571,53.522967
8425,2,7,2,703,2017,12,8,3,3,2,6,1,0,1,0,1,10.025588,53.463445
24881,3,3,53,34,2017,12,8,1,3,9,1,1,0,0,0,2,10.079419,53.283911
24882,3,3,53,34,2017,12,6,1,3,9,1,1,0,0,2,2,10.078947,53.285502
24902,3,1,53,12,2017,12,16,1,2,2,1,1,0,0,2,1,10.140068,51.957632


In [21]:
f"The number of crashes on|close to higways in 2017 is {len(crashData2017_nearby)}"

'The number of crashes on|close to higways in 2017 is 462'

In [12]:
# Calculate distances using a loop
distances = []
for i, row in crashData2018.iterrows():
    lat1, lon1 = row['Latitude'], row['Longitude']
    dist = calculate_distance(np.array([(lat1, lon1)]), weatherData[['Latitude', 'Longitude']].to_numpy())
    distances.append(dist)

# Find matching rows
distances = np.concatenate(distances, axis=0)
distances = distances.reshape(len(crashData2018), len(weatherData))
crashData2018_nearby = crashData2018[np.min(distances, axis=1) <= threshold_distance]

crashData2018_nearby.head()

,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,UTYP1,IstPKW,IstKrad,IstSonstig,LICHT,STRZUSTAND,Longitude,Latitude
6173,2,1,16,136,2018,5,12,5,3,2,6,1,0,0,NaN,0,10.025523,53.516740
6183,2,1,16,136,2018,5,18,7,2,2,6,1,0,0,NaN,0,10.035599,53.495742
6312,2,1,16,136,2018,2,15,6,3,2,6,1,0,0,NaN,1,10.022703,53.476397
6318,2,1,16,136,2018,3,16,5,3,2,6,1,0,0,NaN,0,10.039300,53.502405
6444,2,1,16,136,2018,2,22,6,3,9,1,1,0,0,NaN,0,10.026165,53.484406


In [22]:
f"The number of crashes on|close to higways in 2018 is {len(crashData2018_nearby)}"

'The number of crashes on|close to higways in 2018 is 5404'

In [14]:
# Calculate distances using a loop
distances = []
for i, row in crashData2019.iterrows():
    lat1, lon1 = row['Latitude'], row['Longitude']
    dist = calculate_distance(np.array([(lat1, lon1)]), weatherData[['Latitude', 'Longitude']].to_numpy())
    distances.append(dist)

# Find matching rows
distances = np.concatenate(distances, axis=0)
distances = distances.reshape(len(crashData2019), len(weatherData))
crashData2019_nearby = crashData2019[np.min(distances, axis=1) <= threshold_distance]

crashData2019_nearby.head()

,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,UTYP1,IstPKW,IstKrad,IstSonstig,LICHT,STRZUSTAND,Longitude,Latitude
6167,2,7,2,703,2019,4,14,2,3,2,6,1,0,1,NaN,0,10.024658,53.467342
6373,2,1,16,136,2019,4,6,6,3,3,6,1,0,1,NaN,0,10.020579,53.517460
6508,2,1,15,134,2019,3,23,3,3,8,1,1,0,0,NaN,1,10.027209,53.518536
6614,2,1,16,136,2019,2,20,2,3,2,6,1,0,0,NaN,0,10.032798,53.492378
6742,2,1,16,136,2019,4,14,4,2,2,6,1,0,0,NaN,0,10.034858,53.494853


In [20]:
f"The number of crashes on|close to higways in 2019 is {len(crashData2019_nearby)}"

'The number of crashes on|close to higways in 2019 is 5747'

#### Save CrashData to sql database

In [16]:
crashData2017_nearby.to_sql('CrashData2017_nearby', 'sqlite:///data.sqlite', if_exists='replace')
crashData2018_nearby.to_sql('CrashData2018_nearby', 'sqlite:///data.sqlite', if_exists='replace')
crashData2019_nearby.to_sql('CrashData2019_nearby', 'sqlite:///data.sqlite', if_exists='replace')

5747

#### Concatenate crashData to one dataframe and save to sql database

In [17]:
crashData = pd.concat([crashData2017_nearby, crashData2018_nearby, crashData2019_nearby], ignore_index=True)

#### Match Strecke of weatherData to crashData to assign a strecke value to each crash

In [23]:
# Get the coordinates from both dataframes
crash_coords = crashData[['Latitude', 'Longitude']].values
weather_coords = weatherData[['Latitude', 'Longitude']].values

# Create an empty list to store the corresponding Strecke values
strecke_values = []

# Iterate over each row in crashData
for i, crash_row in crashData.iterrows():
    crash_coord = crash_coords[i]
    distances = calculate_distance(np.array([crash_coord]), weather_coords)
    closest_weather_idx = np.argmin(distances)
    closest_distance = distances[closest_weather_idx]

    # Assign the corresponding Strecke value if the closest distance is within the threshold
    if closest_distance <= threshold_distance:
        strecke_value = weatherData.loc[closest_weather_idx, 'Strecke']
    else:
        strecke_value = None

    strecke_values.append(strecke_value)

# Assign the calculated Strecke values to a new column in crashData
crashData['Strecke'] = strecke_values
crashData.head()

,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,UTYP1,IstPKW,IstKrad,IstSonstig,LICHT,STRZUSTAND,Longitude,Latitude,Strecke
0,2,1,15,134,2017,12,17,4,3,2,6,1,0,0,2.0,1,10.023571,53.522967,Route_Hamburg_Schwieberdingen
1,2,7,2,703,2017,12,8,3,3,2,6,1,0,1,0.0,1,10.025588,53.463445,Route_Hamburg_Schwieberdingen
2,3,3,53,34,2017,12,8,1,3,9,1,1,0,0,0.0,2,10.079419,53.283911,Route_Hamburg_Schwieberdingen
3,3,3,53,34,2017,12,6,1,3,9,1,1,0,0,2.0,2,10.078947,53.285502,Route_Hamburg_Schwieberdingen
4,3,1,53,12,2017,12,16,1,2,2,1,1,0,0,2.0,1,10.140068,51.957632,Route_Hamburg_Schwieberdingen


#### Save CrashData to sql database

In [25]:
crashData.to_sql('CrashData', 'sqlite:///data.sqlite', if_exists='replace')

11613